In [11]:
import tensorflow as tf
import numpy as np

In [12]:
wordsList = np.load('./No_17/training_data/wordsList.npy')
print('loaded the word list!')

loaded the word list!


In [13]:
wordsList = wordsList.tolist() #Originally loaded as numpy array
             #将单词编码为UTF-8格式
wordsList = [word.decode('UTF-8') for word in wordsList]
wordVectors = np.load('./No_17/training_data/wordVectors.npy')
print('Loaded the word vectors!')

Loaded the word vectors!


In [14]:
print(len(wordsList))
print(wordVectors.shape)

400000
(400000, 50)


In [15]:
baseballIndex = wordsList.index('baseball')
wordVectors[baseballIndex]

array([-1.9327  ,  1.0421  , -0.78515 ,  0.91033 ,  0.22711 , -0.62158 ,
       -1.6493  ,  0.07686 , -0.5868  ,  0.058831,  0.35628 ,  0.68916 ,
       -0.50598 ,  0.70473 ,  1.2664  , -0.40031 , -0.020687,  0.80863 ,
       -0.90566 , -0.074054, -0.87675 , -0.6291  , -0.12685 ,  0.11524 ,
       -0.55685 , -1.6826  , -0.26291 ,  0.22632 ,  0.713   , -1.0828  ,
        2.1231  ,  0.49869 ,  0.066711, -0.48226 , -0.17897 ,  0.47699 ,
        0.16384 ,  0.16537 , -0.11506 , -0.15962 , -0.94926 , -0.42833 ,
       -0.59457 ,  1.3566  , -0.27506 ,  0.19918 , -0.36008 ,  0.55667 ,
       -0.70315 ,  0.17157 ], dtype=float32)

In [16]:
#词向量矩阵
maxSeqLength = 10    #单词最大长度
numSeqLength = 300  #单词向量维度 
firstSentence = np.zeros((maxSeqLength),dtype='int32')
firstSentence[0] = wordsList.index('i')
firstSentence[1] = wordsList.index('thought')
firstSentence[2] = wordsList.index('the')
firstSentence[3] = wordsList.index('movie')
firstSentence[4] = wordsList.index('was')
firstSentence[5] = wordsList.index('incredible')
firstSentence[6] = wordsList.index('and')
firstSentence[7] = wordsList.index('inspiring')
firstSentence[8] = wordsList.index('0')
firstSentence[9] = wordsList.index('0')
print(firstSentence.shape)
print(firstSentence)

(10,)
[    41    804 201534   1005     15   7446      5  13767      0      0]


In [17]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
with tf.Session() as sess:
    print(tf.nn.embedding_lookup(wordVectors,firstSentence).eval().shape)
#结果说明输出数据是一个10*50的词矩阵，其中有10个词，每个词的向量维度是50。

(10, 50)


In [18]:
#构造索引之前，可视化所拥有的数据类型，有助于我们决定如何设置最大序列长度的最佳值，

In [19]:
#导入一共25000条电影数据，12500条正向数据和12500条负向数据，并解析这些文件
from os import listdir
from os.path import isfile, join
positiveFiles = ['./No_17/training_data/positiveReviews/' + f for f in listdir('./No_17/training_data/positiveReviews/') if isfile(join('./No_17/training_data/positiveReviews/',f))]
negativeFiles = ['./No_17/training_data/negativeReviews/' + f for f in listdir('./No_17/training_data/negativeReviews/') if isfile(join('./No_17/training_data/negativeReviews/',f))]
numWords=[]
for pf in positiveFiles:
    with open(pf, "r",encoding='utf-8') as f:
        line = f.readline()
        counter = len(line.split())
        numWords.append(counter)
print('Positive files finished')

for nf in negativeFiles:
    with open(nf,"r",encoding='utf-8') as f:
        line= f.readline()
        counter= len(line.split())
        numWords.append(counter)
print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is',numFiles)
print('The average number of words in the files is',sum(numWords)/len(numWords))

Positive files finished
Negative files finished
The total number of files is 25000
The average number of words in the files is 233.7872


In [20]:
#使用matplot对数据进行可视化
#import matplotlib.pyplot as plt
#import matplotlib.font_manager as fm
#myfont = fm.FontProperties(fname='/home/wumg/anaconda3/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/simhei.ttf')
#%matplotlib inline
#plt.hist(numWords,50)
#plt.xlabel('序列长度',fontproperties=myfont)
#plt.ylabel('频度',fontproperties=myfont)
#plt.xaxos([0,1200,0,8000])
#plt.show()

In [21]:
#上述可视化目的是为了找出平均单词，找出合适的句子最大长度，
#下面将单个文件中的文本转换变成索引矩阵。

In [22]:
maxSeqLength = 250
fname = positiveFiles[5] #can use any valid index(not just 3)
with open(fname) as f:
    for lines in f:
        print(lines)
        exit
        

This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br /><

In [23]:
#把他转换成索引矩阵
#删除标点括号问号，只留下字母数字
import re 
strip_special_chars = re.compile("[^A-Za-z0-9]+")

def cleanSentences(string):
    string = string.lower().replace("<br />"," ")
    return re.sub(strip_special_chars,"",string.lower())

firstFile = np.zeros((maxSeqLength),dtype='int32')

with open(fname) as f:
    indexCounter = 0
    line=f.readline()
    cleanedLine = cleanSentences(line)
    split = cleanedLine.split()
    for word in split:
        try:
            firstFile[indexCounter] = wordsList.index(word)
        except ValueError:
            firstFile[indexCounter] = 399999 #Vector for unknown words
        indexCounter = indexCounter + 1
firstFile

array([399999,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [24]:
#利用相同方法，处理全部25000条论文，导入电影训练集，并得到25000x250的矩阵，采用迁移学习的方法，直接使用训练好的索引矩阵文件，
ids = np.load('./No_17/training_Data/idsMatrix.npy')

In [25]:
#定义辅助函数，在训练神经网络步骤需要

from random import randint

def getTrainBatch():
    labels= []
    arr = np.zeros([batchSize,MaxSeqLength])
    for i in range(batchSize):
        if (i%2 == 0 ):
            num = randint(1,11499)
            labels.append([1,0])
        else:
            num = randint(13299,24999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr,labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize,maxSeqLength])
    for i in range(batchSize):
        num = randint(11499,13499)
        if (num <=12499 ):
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr,labels


In [26]:
#构建RNN模型，在构建图模型前，先定义一个超参数，批处理大小，LSTM单元个数，分类类别，训练次数
batchSize=24
lstmUnits=64
numClasses = 2
iterations = 20000

In [27]:
#指定两个占位符，一个用于数据输入，另一个用来标签数据，
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
tf.compat.v1.disable_eager_execution()
ops.reset_default_graph()
labels = tf.placeholder(tf.float32,[batchSize,numClasses])
input_data = tf.placeholder(tf.int32,[batchSize,maxSeqLength])


In [28]:
data = tf.nn.embedding_lookup(wordVectors,input_data)

In [33]:
#构建RNN模型
#因为教程使用tf1.6所以此段构建RNN模型函数失败。
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# -outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
outputs, states = tf.compat.v1.nn.static_rnn(lstm_cells, _X, dtype=tf.float32)
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell,output_keep_prob=0.25)
value,_ = tf.nn.dynamic_rnn(lstmCell,data,dtype=tf.float32)

AttributeError: module 'tensorflow_core.compat.v1' has no attribute 'contrib'

In [34]:
#dynamic RNN 函数第一个输出被认为是最后隐藏状态向量，被重新定义维度，然后乘以最高的权重矩阵和偏置项后获得最终输出。
weight = tf.Variable(tf.truncated_narmal([lstmUnits,numClasses]))
biias = tf.Variable(tf.constant(0.1,shape=[numClasses]))
value = tf.transpose(value,[1,0,2])
last = tf.gather(value,int(valu.get_shape()[0])-1)
prediction = (tf.matmul(last,weight)+bias)

AttributeError: module 'tensorflow_core.compat.v1' has no attribute 'truncated_narmal'

In [ ]:
correctPred = tf.equal(tf.argmax(prediction,1),tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred,tf.float32))

In [ ]:
loss = tf.reduce_mean(tf.nn.soft_cross_entropy_with_logits_v2(logits=prediction,labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


In [ ]:
#使用tensorboard来可视化损失值和准确率
import datetime
tf.summary.scalar('Loss',loss)
tf.summary.scalar('Accuracy',accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +"/"
writer = tf.summary.FileWriter(logdir,sess.graph)

In [ ]:
#模型训练
#定义一个Tensorflow 会话，然后加入一批评论和对应的标签，
#调用run函数，run函数有两个参数，第一个叫fetches，定义了我们感兴趣的值，希望通过优化器来优化最小代价函数，
#第二个函数叫feed_dict参数，这个数据结构就是占位符，
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
with tf.device('/gpu:0'):
    for i in range(iterations):
        #获取下一批数据
        nextBatch,nextBatchLabels = getTrainBatch();
        sess.run(optimizer,{input_data:nextBatch,labels:nextBatchLabels})
        #汇总信息写入Tensorboard
        if (i%50 == 0):
            summary = sess.run(merged,{input_data:nextBatch,labels:nextBatchLabels})
            writer.add_summary(summary,i)
        #每训练100次保存一次
        if (i/1000 == 0 and i != 0):
            save_path = saver.save(sess,"models/pretrained_lstm.ckpt",global_step=i)
            print("saved to %s" % save_path)
    writer.close()
    

In [ ]:
#通过代码查看批处理的准确性，
 iterations = 10
    for i in range(iterations):
        nextBatch,nextBatchLabels = getTestBatch();
        print("Accuracy for this batch:",(sess.run(accuracy,{input_data:nextBatch,labels:nextBatchLabels}))*100)